In [154]:
%matplotlib inline

import random

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from scipy import stats
from scipy.stats import randint, uniform
from sklearn import linear_model, preprocessing
from sklearn.decomposition import PCA
from sklearn.ensemble import (AdaBoostClassifier, BaggingClassifier,
                              RandomForestClassifier, RandomForestRegressor)
from sklearn.linear_model import Lasso, LogisticRegression, Ridge
from sklearn.metrics import (accuracy_score, classification_report,
                             confusion_matrix, mean_absolute_error, r2_score)
from sklearn.model_selection import (GridSearchCV, RandomizedSearchCV,
                                     train_test_split)
from sklearn.preprocessing import (LabelEncoder, MinMaxScaler, OneHotEncoder,
                                   PolynomialFeatures, RobustScaler,
                                   StandardScaler)
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from skopt import BayesSearchCV

pd.set_option('display.max_rows',1000)
pd.set_option('display.max_columns',1000)

In [155]:
# import data
dataset_og = pd.read_csv('Data\Measurements-Transformed')
# kopie maken indien we iets van de originele data nodig hebben
dataset = dataset_og.copy()
dataset.head(5)

,ID,Sex,Measurement_Age,Add,Sph-Far-R,Cyl-Far-R,Axis-Far-R,Sph-Close-R,Cyl-Close-R,Axis-Close-R,Sph-Far-L,Cyl-Far-L,Axis-Far-L,Sph-Close-L,Cyl-Close-L,Axis-Close-L
0,371435.0,0.0,21118.0,0.0,-1.75,0.50,55.0,-2.25,1.0,55.0,-1.75,1.00,110.0,-1.25,0.5,110.0
1,371435.0,0.0,20245.0,0.0,-1.75,0.50,65.0,0.00,0.0,0.0,-1.25,0.50,110.0,0.00,0.0,0.0
2,371435.0,0.0,18099.0,0.0,-1.50,0.50,65.0,0.00,0.0,0.0,-1.00,0.50,110.0,0.00,0.0,0.0
3,402916.0,1.0,13825.0,0.0,-3.50,1.50,180.0,-3.50,1.5,180.0,-3.00,1.50,180.0,-3.00,1.5,180.0
4,402916.0,1.0,9653.0,0.0,-2.00,0.75,175.0,0.00,0.0,0.0,-2.00,0.75,180.0,0.00,0.0,0.0


In [156]:
#drop rijen waar < n meting van zijn en houd van de overige de top n meest recente waardes
dataset = dataset.groupby('ID').filter(lambda x: len(x) > 1)
dataset = dataset.groupby('ID').head(2)

In [157]:
dataset.head()

,ID,Sex,Measurement_Age,Add,Sph-Far-R,Cyl-Far-R,Axis-Far-R,Sph-Close-R,Cyl-Close-R,Axis-Close-R,Sph-Far-L,Cyl-Far-L,Axis-Far-L,Sph-Close-L,Cyl-Close-L,Axis-Close-L
0,371435.0,0.0,21118.0,0.0,-1.75,0.50,55.0,-2.25,1.0,55.0,-1.75,1.00,110.0,-1.25,0.50,110.0
1,371435.0,0.0,20245.0,0.0,-1.75,0.50,65.0,0.00,0.0,0.0,-1.25,0.50,110.0,0.00,0.00,0.0
3,402916.0,1.0,13825.0,0.0,-3.50,1.50,180.0,-3.50,1.5,180.0,-3.00,1.50,180.0,-3.00,1.50,180.0
4,402916.0,1.0,9653.0,0.0,-2.00,0.75,175.0,0.00,0.0,0.0,-2.00,0.75,180.0,0.00,0.00,0.0
7,662712.0,0.0,25627.0,3.0,2.25,0.75,90.0,5.25,1.0,90.0,2.75,1.00,95.0,5.75,0.75,95.0


In [158]:
#2 rijen naast elkaar zetten

dataset = dataset.merge(dataset ,on=['ID', 'Sex'], suffixes=['_x', ''])
dataset = dataset.sort_values(by=['ID', 'Measurement_Age_x'])
dataset = dataset.drop_duplicates(subset=['ID', 'Sex'], keep='first')
dataset.head(20)
# dataset.loc[dataset['ID'] == 471318.0]

,ID,Sex,Measurement_Age_x,Add_x,Sph-Far-R_x,Cyl-Far-R_x,Axis-Far-R_x,Sph-Close-R_x,Cyl-Close-R_x,Axis-Close-R_x,Sph-Far-L_x,Cyl-Far-L_x,Axis-Far-L_x,Sph-Close-L_x,Cyl-Close-L_x,Axis-Close-L_x,Measurement_Age,Add,Sph-Far-R,Cyl-Far-R,Axis-Far-R,Sph-Close-R,Cyl-Close-R,Axis-Close-R,Sph-Far-L,Cyl-Far-L,Axis-Far-L,Sph-Close-L,Cyl-Close-L,Axis-Close-L
19490,100063.0,1.0,11602.0,0.00,-2.75,1.00,105.0,0.00,0.00,0.0,-3.50,1.75,90.0,0.00,0.00,0.0,13181.0,0.00,-2.75,1.00,105.0,-2.75,1.00,105.0,-3.50,1.75,90.0,-3.50,1.75,90.0
28452,100167.0,0.0,15213.0,0.00,0.75,0.00,0.0,0.75,0.00,0.0,0.75,0.00,0.0,0.75,0.00,0.0,15213.0,0.00,0.75,0.00,0.0,0.75,0.00,0.0,0.75,0.00,0.0,0.75,0.00,0.0
102,100238.0,1.0,20731.0,0.00,0.00,0.00,0.0,1.50,0.50,75.0,0.00,0.00,0.0,1.75,0.50,105.0,23791.0,0.00,2.00,0.50,75.0,2.00,0.50,75.0,2.25,0.50,105.0,2.25,0.50,105.0
23794,100262.0,0.0,17107.0,0.00,0.00,0.00,0.0,1.25,0.00,0.0,0.00,0.00,0.0,1.75,0.00,0.0,19170.0,0.00,1.50,0.00,0.0,1.50,0.00,0.0,2.00,0.00,0.0,2.00,0.00,0.0
38518,100315.0,0.0,1757.0,0.00,2.00,0.00,0.0,0.00,0.00,0.0,3.00,0.00,0.0,0.00,0.00,0.0,2075.0,0.00,2.50,0.00,0.0,2.50,0.00,0.0,3.25,0.00,0.0,3.25,0.00,0.0
37422,100320.0,1.0,21477.0,2.00,3.50,0.50,160.0,5.50,0.50,160.0,3.50,0.50,40.0,5.50,0.50,40.0,22573.0,2.25,4.00,0.25,125.0,6.25,0.25,125.0,4.00,0.25,60.0,6.25,0.25,60.0
36454,100341.0,1.0,19241.0,0.00,-3.25,0.00,0.0,0.00,0.00,0.0,-3.00,0.00,0.0,0.00,0.00,0.0,19271.0,0.00,-3.25,0.00,0.0,-3.25,0.00,0.0,-3.00,0.00,0.0,-3.00,0.00,0.0
19934,100348.0,0.0,18753.0,0.00,-2.25,1.50,85.0,0.00,0.00,0.0,-1.00,0.50,90.0,0.00,0.00,0.0,21449.0,0.00,0.75,0.50,85.0,0.75,0.50,85.0,0.00,1.50,90.0,0.00,1.50,90.0
43096,100358.0,1.0,26654.0,0.00,0.00,0.00,0.0,3.25,1.50,180.0,0.00,0.00,0.0,2.75,1.75,180.0,26654.0,0.00,0.00,0.00,0.0,3.25,1.50,180.0,0.00,0.00,0.0,2.75,1.75,180.0
42470,100442.0,0.0,22986.0,2.25,2.00,0.00,0.0,4.25,0.00,0.0,2.00,0.00,0.0,4.25,0.00,0.0,24684.0,2.75,2.00,0.50,90.0,4.75,0.50,90.0,2.00,0.50,180.0,4.75,0.50,180.0


In [159]:
#Drop kolom ID
dataset.drop(['ID'],axis=1, inplace=True)


### Linear regression

In [160]:
# Splitsen in features en targets

y = dataset['Sph-Far-R'].values
X = dataset.drop(['Add', 'Sph-Far-R', 'Cyl-Far-R', 'Axis-Far-R', 'Sph-Close-R', 'Cyl-Close-R', 'Axis-Close-R', 'Sph-Far-L', 
                  'Cyl-Far-L', 'Axis-Far-L', 'Sph-Close-L', 'Cyl-Close-L', 'Axis-Close-L'],axis=1)

# Splitsen in training set en test set

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)


# MinMax scaling

scaler = MinMaxScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

# Linear regression

lreg_sph_far_r = linear_model.LinearRegression()
lreg_sph_far_r.fit(X_train,y_train)
print(lreg_sph_far_r.coef_)
r2 = lreg_sph_far_r.score(X_test,y_test)
print('r2 score = ', r2)

# Modeloptimalisatie en Hyperparameter tuning

# Aanmaken van de hogere orde features
graad = 3

poly = PolynomialFeatures(graad)
poly.fit(X_train)
X_train_poly = poly.transform(X_train)
X_test_poly = poly.transform(X_test)
print('dimensie van X_train_poly: ',X_train_poly.shape)
print('dimensie van X_test_poly: ',X_test_poly.shape)


# met L2 regularisatie via Ridge regression
lreg_poly_sph_far_r = Ridge(alpha=0.01,tol=0.0001,fit_intercept=True)
lreg_poly_sph_far_r.fit(X_train_poly,y_train)

print('R2 score op test set via L2: ',lreg_poly_sph_far_r.score(X_test_poly,y_test))
# R2 -score via L2 op de trainingset
print('R2 score op training set via L2: ',lreg_poly_sph_far_r.score(X_train_poly,y_train))


# # met L1 regularisatie via Lasso regression
# lreg_poly_sph_far_r = Lasso(alpha=0.001,tol=0.0001,fit_intercept=True)
# lreg_poly_sph_far_r.fit(X_train_poly,y_train)      
  
      
# print('R2 score op test set via L1: ',lreg_poly_sph_far_r.score(X_test_poly,y_test))
  
# # R2 -score via L1 op de trainingset
# print('R2 score op training set via L1: ',lreg_poly_sph_far_r.score(X_train_poly,y_train))    

# # Variëren van de alpha en grafiek

# train_r2 = []
# test_r2 = []


# alphas = np.logspace(-2, 6, 1000)

# for alpha in alphas:
#     lregmodel_poly = Ridge(alpha=alpha,tol=0.0001,fit_intercept=True)
#     lregmodel_poly.fit(X_train_poly,y_train)
#     test_r2.append(lregmodel_poly.score(X_test_poly,y_test))  
#     train_r2.append(lregmodel_poly.score(X_train_poly,y_train))


# # Plot r2
# f, ax = plt.subplots(figsize=(10, 8))
# plt.subplot(2, 1, 1)
# plt.semilogx(alphas, train_r2, label='Train')
# plt.semilogx(alphas, test_r2, label='Test')
# plt.legend(loc='lower left')
# plt.ylim([0, 1.2])
# plt.xlabel('Regularization parameter')
# plt.ylabel('R² Performance')



# Via KernelRidge met een polynomial kernel

from sklearn.kernel_ridge import KernelRidge

lreg_sph_far_r = KernelRidge(alpha=0.001, degree=3, gamma=None, kernel='polynomial')
lreg_sph_far_r.fit(X_train,y_train)


y_predicted = lreg_sph_far_r.predict(X_train)
r2 = r2_score(y_train,y_predicted)

print('training set: ',r2)

y_predicted = lreg_sph_far_r.predict(X_test)
r2 = r2_score(y_test,y_predicted)

print('test set: ',r2)

[ 3.50335677e-02 -1.11394914e+00 -5.02890242e-01  8.01063463e+01
  3.30568057e-01 -2.98768530e-01  6.96003079e+00 -1.44271433e-01
  2.44420702e-01  2.02487582e+00  1.22962007e-01 -2.74213260e-01
 -2.05821244e-01 -2.49127767e-01  2.39545826e-01  2.76995835e+00]
r2 score =  0.7600462313709082
dimensie van X_train_poly:  (8758, 969)
dimensie van X_test_poly:  (2190, 969)
R2 score op test set via L2:  0.7788684126121043
R2 score op training set via L2:  0.8322479307924304
training set:  0.8148386471820034
test set:  0.7794425067469433


In [161]:
# Splitsen in features en targets

y = dataset['Cyl-Far-R'].values
X = dataset.drop(['Add', 'Sph-Far-R', 'Cyl-Far-R', 'Axis-Far-R', 'Sph-Close-R', 'Cyl-Close-R', 'Axis-Close-R', 'Sph-Far-L', 
                  'Cyl-Far-L', 'Axis-Far-L', 'Sph-Close-L', 'Cyl-Close-L', 'Axis-Close-L'],axis=1)

# Splitsen in training set en test set

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)


# MinMax scaling

scaler = MinMaxScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

# Linear regression

lreg_cyl_far_r = linear_model.LinearRegression()
lreg_cyl_far_r.fit(X_train,y_train)
print(lreg_cyl_far_r.coef_)
r2 = lreg_cyl_far_r.score(X_test,y_test)
print('r2 score = ', r2)

# Modeloptimalisatie en Hyperparameter tuning

# Aanmaken van de hogere orde features
graad = 2

poly = PolynomialFeatures(graad)
poly.fit(X_train)
X_train_poly = poly.transform(X_train)
X_test_poly = poly.transform(X_test)
print('dimensie van X_train_poly: ',X_train_poly.shape)
print('dimensie van X_test_poly: ',X_test_poly.shape)


# met L2 regularisatie via Ridge regression
lreg_poly_cyl_far_r = Ridge(alpha=0.5,tol=0.0001,fit_intercept=True)
lreg_poly_cyl_far_r.fit(X_train_poly,y_train)

print('R2 score op test set via L2: ',lreg_poly_cyl_far_r.score(X_test_poly,y_test))
# R2 -score via L2 op de trainingset
print('R2 score op training set via L2: ',lreg_poly_cyl_far_r.score(X_train_poly,y_train))

[ 0.02754036 -1.08471856 -0.13591271 -0.71844359  1.77899019 -0.13386286
  0.4141191   0.39415392  0.1231833   0.02745607  0.27202712 -0.01021161
  0.19649597 -0.15741287  0.02460432  1.18918183]
r2 score =  0.5887542128275896
dimensie van X_train_poly:  (8758, 153)
dimensie van X_test_poly:  (2190, 153)
R2 score op test set via L2:  0.6321228097464662
R2 score op training set via L2:  0.6461088211673364


In [162]:
# Splitsen in features en targets

y = dataset['Sph-Far-L'].values
X = dataset.drop(['Add', 'Sph-Far-R', 'Cyl-Far-R', 'Axis-Far-R', 'Sph-Close-R', 'Cyl-Close-R', 'Axis-Close-R', 'Sph-Far-L', 
                  'Cyl-Far-L', 'Axis-Far-L', 'Sph-Close-L', 'Cyl-Close-L', 'Axis-Close-L'],axis=1)

# Splitsen in training set en test set

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)


# MinMax scaling

scaler = MinMaxScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

# Linear regression

lreg_sph_far_l = linear_model.LinearRegression()
lreg_sph_far_l.fit(X_train,y_train)
print(lreg_sph_far_l.coef_)
r2 = lreg_sph_far_l.score(X_test,y_test)
print('r2 score = ', r2)

# Modeloptimalisatie en Hyperparameter tuning

# Aanmaken van de hogere orde features
graad = 3

poly = PolynomialFeatures(graad)
poly.fit(X_train)
X_train_poly = poly.transform(X_train)
X_test_poly = poly.transform(X_test)
print('dimensie van X_train_poly: ',X_train_poly.shape)
print('dimensie van X_test_poly: ',X_test_poly.shape)


# met L2 regularisatie via Ridge regression
lreg_poly_sph_far_l = Ridge(alpha=0.5,tol=0.0001,fit_intercept=True)
lreg_poly_sph_far_l.fit(X_train_poly,y_train)

print('R2 score op test set via L2: ',lreg_poly_sph_far_l.score(X_test_poly,y_test))
# R2 -score via L2 op de trainingset
print('R2 score op training set via L2: ',lreg_poly_sph_far_l.score(X_train_poly,y_train))

[ 1.70819721e-02 -1.11076253e+00 -5.11045324e-01  9.13057334e+00
 -9.82420845e-03 -3.79148562e-01 -5.92690046e+00 -6.23964856e-02
  3.79331195e-01  2.79072402e+01  3.38812244e-01 -1.54120062e-01
  3.28711291e+00 -2.63459890e-01  9.42695818e-02  2.75525553e+00]
r2 score =  0.7558198646846204
dimensie van X_train_poly:  (8758, 969)
dimensie van X_test_poly:  (2190, 969)
R2 score op test set via L2:  0.7745897043364096
R2 score op training set via L2:  0.8063006630714489


In [163]:
# Splitsen in features en targets

y = dataset['Cyl-Far-L'].values
X = dataset.drop(['Add', 'Sph-Far-R', 'Cyl-Far-R', 'Axis-Far-R', 'Sph-Close-R', 'Cyl-Close-R', 'Axis-Close-R', 'Sph-Far-L', 
                  'Cyl-Far-L', 'Axis-Far-L', 'Sph-Close-L', 'Cyl-Close-L', 'Axis-Close-L'],axis=1)

# Splitsen in training set en test set

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

# MinMax scaling

scaler = MinMaxScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

# Linear regression

lreg_cyl_far_l = linear_model.LinearRegression()
lreg_cyl_far_l.fit(X_train,y_train)
print(lreg_cyl_far_l.coef_)
r2 = lreg_cyl_far_l.score(X_test,y_test)
print('r2 score = ', r2)

# Modeloptimalisatie en Hyperparameter tuning

# Aanmaken van de hogere orde features
graad = 2

poly = PolynomialFeatures(graad)
poly.fit(X_train)
X_train_poly = poly.transform(X_train)
X_test_poly = poly.transform(X_test)
print('dimensie van X_train_poly: ',X_train_poly.shape)
print('dimensie van X_test_poly: ',X_test_poly.shape)


# met L2 regularisatie via Ridge regression
lreg_poly_cyl_far_l = Ridge(alpha=0.5,tol=0.0001,fit_intercept=True)
lreg_poly_cyl_far_l.fit(X_train_poly,y_train)

print('R2 score op test set via L2: ',lreg_poly_cyl_far_l.score(X_test_poly,y_test))
# R2 -score via L2 op de trainingset
print('R2 score op training set via L2: ',lreg_poly_cyl_far_l.score(X_train_poly,y_train))

[ 0.01846271 -1.19452618 -0.18611335 -0.32827664  0.2146676   0.01242735
  0.13317797 -0.12753369  0.00751347 -0.16688369  1.847539   -0.13201424
  0.40134644  0.48429999  0.11467518  1.34400184]
r2 score =  0.6200897564008739
dimensie van X_train_poly:  (8758, 153)
dimensie van X_test_poly:  (2190, 153)
R2 score op test set via L2:  0.6506316440314254
R2 score op training set via L2:  0.6667621222476299


### Random forrest regressor

In [164]:
# Splitsen in features en targets

y = dataset['Sph-Far-R'].values
X = dataset.drop(['Add', 'Sph-Far-R', 'Cyl-Far-R', 'Axis-Far-R', 'Sph-Close-R', 'Cyl-Close-R', 'Axis-Close-R', 'Sph-Far-L', 
                  'Cyl-Far-L', 'Axis-Far-L', 'Sph-Close-L', 'Cyl-Close-L', 'Axis-Close-L'],axis=1)

# Splitsen in training set en test set

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

#random forest regressor
RFR_model_sph_far_r = RandomForestRegressor(n_estimators=200)
RFR_model_sph_far_r.fit(X_train,y_train)

RFR_model_sph_far_r.score(X_test,y_test)

0.7868072505852319

In [165]:
# Splitsen in features en targets

y = dataset['Cyl-Far-R'].values
X = dataset.drop(['Add', 'Sph-Far-R', 'Cyl-Far-R', 'Axis-Far-R', 'Sph-Close-R', 'Cyl-Close-R', 'Axis-Close-R', 'Sph-Far-L', 
                  'Cyl-Far-L', 'Axis-Far-L', 'Sph-Close-L', 'Cyl-Close-L', 'Axis-Close-L'],axis=1)

# Splitsen in training set en test set

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)


#random forest regressor
RFR_model_cyl_far_r = RandomForestRegressor(n_estimators=200)
RFR_model_cyl_far_r.fit(X_train,y_train)

RFR_model_cyl_far_r.score(X_test,y_test)

0.5970506057560836

In [166]:
# Splitsen in features en targets

y = dataset['Sph-Far-L'].values
X = dataset.drop(['Add', 'Sph-Far-R', 'Cyl-Far-R', 'Axis-Far-R', 'Sph-Close-R', 'Cyl-Close-R', 'Axis-Close-R', 'Sph-Far-L', 
                  'Cyl-Far-L', 'Axis-Far-L', 'Sph-Close-L', 'Cyl-Close-L', 'Axis-Close-L'],axis=1)

# Splitsen in training set en test set

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)


#random forest regressor
RFR_model_sph_far_l = RandomForestRegressor(n_estimators=200)
RFR_model_sph_far_l.fit(X_train,y_train)

RFR_model_sph_far_l.score(X_test,y_test)

0.7922091382945503

In [167]:
# Splitsen in features en targets

y = dataset['Cyl-Far-L'].values
X = dataset.drop(['Add', 'Sph-Far-R', 'Cyl-Far-R', 'Axis-Far-R', 'Sph-Close-R', 'Cyl-Close-R', 'Axis-Close-R', 'Sph-Far-L', 
                  'Cyl-Far-L', 'Axis-Far-L', 'Sph-Close-L', 'Cyl-Close-L', 'Axis-Close-L'],axis=1)

# Splitsen in training set en test set

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)


#random forest regressor
RFR_model_cyl_far_l = RandomForestRegressor(n_estimators=200)
RFR_model_cyl_far_l.fit(X_train,y_train)

RFR_model_cyl_far_l.score(X_test,y_test)

0.6258113614458625

In [168]:
dataset.head()

,Sex,Measurement_Age_x,Add_x,Sph-Far-R_x,Cyl-Far-R_x,Axis-Far-R_x,Sph-Close-R_x,Cyl-Close-R_x,Axis-Close-R_x,Sph-Far-L_x,Cyl-Far-L_x,Axis-Far-L_x,Sph-Close-L_x,Cyl-Close-L_x,Axis-Close-L_x,Measurement_Age,Add,Sph-Far-R,Cyl-Far-R,Axis-Far-R,Sph-Close-R,Cyl-Close-R,Axis-Close-R,Sph-Far-L,Cyl-Far-L,Axis-Far-L,Sph-Close-L,Cyl-Close-L,Axis-Close-L
19490,1.0,11602.0,0.0,-2.75,1.0,105.0,0.00,0.0,0.0,-3.50,1.75,90.0,0.00,0.0,0.0,13181.0,0.0,-2.75,1.0,105.0,-2.75,1.0,105.0,-3.50,1.75,90.0,-3.50,1.75,90.0
28452,0.0,15213.0,0.0,0.75,0.0,0.0,0.75,0.0,0.0,0.75,0.00,0.0,0.75,0.0,0.0,15213.0,0.0,0.75,0.0,0.0,0.75,0.0,0.0,0.75,0.00,0.0,0.75,0.00,0.0
102,1.0,20731.0,0.0,0.00,0.0,0.0,1.50,0.5,75.0,0.00,0.00,0.0,1.75,0.5,105.0,23791.0,0.0,2.00,0.5,75.0,2.00,0.5,75.0,2.25,0.50,105.0,2.25,0.50,105.0
23794,0.0,17107.0,0.0,0.00,0.0,0.0,1.25,0.0,0.0,0.00,0.00,0.0,1.75,0.0,0.0,19170.0,0.0,1.50,0.0,0.0,1.50,0.0,0.0,2.00,0.00,0.0,2.00,0.00,0.0
38518,0.0,1757.0,0.0,2.00,0.0,0.0,0.00,0.0,0.0,3.00,0.00,0.0,0.00,0.0,0.0,2075.0,0.0,2.50,0.0,0.0,2.50,0.0,0.0,3.25,0.00,0.0,3.25,0.00,0.0


In [169]:
values = np.array([1, 11602.0, 0.0, -2.75, 1.0, 105.0, 0.00, 0.0, 0.0, -3.50, 1.75, 90.0, 0.00, 0.0, 0.0, 13181.0])
#Sph-Far-R = -2.75
#Cyl-Far-R = 1.0
#Sph-Far-L = -3.5
#Cyl-Far-L = 1.75

values = values.reshape(1,-1)
# scaler = MinMaxScaler()
# scaler.fit(values)
# values = scaler.transform(values)

models = [lreg_sph_far_r, lreg_cyl_far_r, lreg_sph_far_l, lreg_cyl_far_l,
          RFR_model_sph_far_r, RFR_model_cyl_far_r, RFR_model_sph_far_l, RFR_model_cyl_far_l]

for n in models:
    x = n.predict(values)
    print(x)


[-7.02891421e+12]
[3078.94657058]
[23234.70919553]
[3850.78127162]
[-2.81375]
[0.98875]
[-3.5425]
[1.82625]
